In [1]:
!pip install pycorenlp

In [36]:
import html
import re

In [37]:
import pandas as pd 

data = pd.read_csv('Data/twitter_corpus-master/full-corpus.csv')

In [38]:
data.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [39]:
data.Sentiment.unique()

array(['positive', 'negative', 'neutral', 'irrelevant'], dtype=object)

In [40]:
test = data[data.Sentiment != 'irrelevant']
test.drop(['Topic', 'TweetId', 'TweetDate'], inplace=True, axis=1)

In [41]:
test.loc[:, 'TweetText'] = test.loc[:, 'TweetText'].apply(lambda x: html.unescape(x))
test.loc[:, 'TweetText'] = test.loc[:, 'TweetText'].apply(lambda x: re.sub(r'(www\.|https?://).*?(\s|$)|@.*?(\s|$)|\$.*?(\s|$)|\d|\%|\\|/|-|_', ' ', x))
test.loc[:, 'TweetText'] = test.loc[:, 'TweetText'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [42]:
test.head()

,Sentiment,TweetText
0,positive,Now all has to do is get swype on the iphone a...
1,positive,will be adding more carrier support to the iP...
2,positive,Hilarious video guy does a duet with 's Siri. ...
3,positive,you made it too easy for me to switch to iPho...
4,positive,I just realized that the reason I got into twi...


In [43]:
from pycorenlp import StanfordCoreNLP
import time

nlp = StanfordCoreNLP('http://localhost:9000')

In [71]:
scores = []
ss = 0
length_s = 0

start = time.process_time() 
for i in range(len(test)):
    text = test['TweetText'][i]
    result = nlp.annotate(text,
                   properties={
                       'annotators': 'sentiment, ner, pos',
                       'outputFormat': 'json',
                       'timeout': 1000,
                   })

    for s in result['sentences']:
        ss += int(s['sentimentValue'])
        length_s += 1
        
    if (length_s > 0):
        score = ss/length_s
    if (score < 2):
        score_text = 'negative'
    elif(score == 2):
        score_text = 'neutral'
    else: 
        score_text = 'positive'
        
    scores.append(score_text)

time_CoreNLP = time.process_time() - start

KeyError: 1003

In [4]:
len(scores)

data.iloc[460,:]

sentiments = data['Sentiment'][0:460]
index = 0
correct_counter = 0
for x in sentiments:
    if x == scores[index]:
        correct_counter = correct_counter + 1
    
print(correct_counter/461)

IndexError: list index out of range